# __Análise de Rede Social: Motociada de 14 de abril de 2022, São Paulo__

## Coleta de Dados

In [1]:
# imports
import sys
sys.path.insert(1, '../')

from credentials import *
import tweepy as tw
import pandas as pd

In [2]:
# autenticação
auth = tw.OAuth1UserHandler(API_KEY, API_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tw.API(auth, wait_on_rate_limit=True)

In [3]:
# hashtags selecionadas para a coleta
hashtags = ['#VaiSerNoPrimeiroTurno', '#BolsonaroReeleitoEm2022', '#MaiorMotociataDoMundo', '#Vagabundociata']
query = ' OR '.join(hashtags)

# dias selecionados para a coleta
since = '2022-04-14'
until = '2022-04-21'

In [4]:
# coleta de tweets
tweets = [tweet for tweet in tw.Cursor(api.search_tweets,
                                      q=query,
                                      lang='pt',
                                      until=until,
                                      result_type='mixed',
                                      tweet_mode='extended').items()]

Rate limit reached. Sleeping for: 793
Rate limit reached. Sleeping for: 788
Rate limit reached. Sleeping for: 787
Rate limit reached. Sleeping for: 794


In [27]:
len(tweets)

12742

In [67]:
# antes de carregar o arquivo json eu salvei os tweets coletados nesse mesmo arquivo
tweets_json = json.load(open('data.json'))

In [68]:
tweets_json[1]

{'created_at': 'Fri Apr 15 21:16:26 +0000 2022',
 'id': 1515076605596651521,
 'id_str': '1515076605596651521',
 'text': '@LulaOficial Sentiu!!! #MaiorMotociataDoMundo',
 'truncated': False,
 'entities': {'hashtags': [{'text': 'MaiorMotociataDoMundo',
    'indices': [23, 45]}],
  'symbols': [],
  'user_mentions': [{'screen_name': 'LulaOficial',
    'name': 'Lula',
    'id': 2670726740,
    'id_str': '2670726740',
    'indices': [0, 12]}],
  'urls': []},
 'metadata': {'iso_language_code': 'pt', 'result_type': 'popular'},
 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 'in_reply_to_status_id': 1514996929414979587,
 'in_reply_to_status_id_str': '1514996929414979587',
 'in_reply_to_user_id': 2670726740,
 'in_reply_to_user_id_str': '2670726740',
 'in_reply_to_screen_name': 'LulaOficial',
 'user': {'id': 40053694,
  'id_str': '40053694',
  'name': 'Flavio Bolsonaro',
  'screen_name': 'FlavioBolsonaro',
  'location': 'Rio de Janeiro',
  'descri

In [10]:
columns = ['created_at', 'tweet_content', 'truncated_status', 'tweet_metadata', 'author', 'author_metadata',
           'hashtags', 'is_retweet', 'is_quoted_rt', 'is_reply', 'user_mentions', 'retweeted_from', 'quoted_from', 'in_reply_to']

In [28]:
tt_data = []

for tweet in tweets_json:
    created_at = tweet['created_at']
    tweet_content = tweet['text']
    truncated = tweet['truncated']
    tweet_metadata = {'tweet_id': tweet['id'],
                      'like_count': tweet['favorite_count'],
                      'retweet_count': tweet['retweet_count']}
    author = tweet['user']['screen_name']
    author_metadata = {'user_id': tweet['user']['id'],
                       'name': tweet['user']['name'],
                       'description': tweet['user']['description'],
                       'verified_status': tweet['user']['verified'],
                       'created_at': tweet['user']['created_at']}
    hashtags = [hashtag['text'] for hashtag in tweet['entities']['hashtags']]
    is_retweet = True if 'retweeted_status' in tweet.keys() else False
    is_quoted_rt = True if 'quoted_status' in tweet.keys() else False
    is_reply = True if tweet['in_reply_to_user_id'] != None else False
    user_mentions = [mention['screen_name'] for mention in tweet['entities']['user_mentions']] if len(tweet['entities']['user_mentions']) > 0 else None
    retweeted_from = tweet['retweeted_status']['user']['screen_name'] if is_retweet else None
    quoted_from = tweet['quoted_status']['user']['screen_name'] if is_quoted_rt else None
    in_reply_to = None if not is_reply else tweet['in_reply_to_screen_name']
    tt_data.append((created_at, tweet_content, truncated, tweet_metadata, author, author_metadata, hashtags, is_retweet, is_quoted_rt, is_reply,
                    user_mentions, retweeted_from, quoted_from, in_reply_to))

tt_df = pd.DataFrame(columns=columns, data=tt_data)
tt_df.head()

,created_at,tweet_content,truncated_status,tweet_metadata,author,author_metadata,hashtags,is_retweet,is_quoted_rt,is_reply,user_mentions,retweeted_from,quoted_from,in_reply_to
0,Fri Apr 15 19:12:02 +0000 2022,"Imprensa, NÃO BRIGUE COM A IMAGEM\n#MaiorMotoc...",False,"{'tweet_id': 1515045299835838467, 'like_count'...",taoquei1,"{'user_id': 1087259768, 'name': 'TeAtualizei 🇧...",[MaiorMotociataDoMundo],False,False,False,None,None,None,None
1,Fri Apr 15 21:16:26 +0000 2022,@LulaOficial Sentiu!!! #MaiorMotociataDoMundo,False,"{'tweet_id': 1515076605596651521, 'like_count'...",FlavioBolsonaro,"{'user_id': 40053694, 'name': 'Flavio Bolsonar...",[MaiorMotociataDoMundo],False,False,True,[LulaOficial],None,None,LulaOficial
2,Sat Apr 16 13:16:16 +0000 2022,O jingle da #ViradaRJ não sai da minha cabeça ...,True,"{'tweet_id': 1515318156977491972, 'like_count'...",apellegrino,"{'user_id': 2689114595, 'name': 'Antonia Pelle...",[ViradaRJ],False,False,False,[MarceloFreixo],None,None,None
3,Wed Apr 20 23:59:57 +0000 2022,@senadorhumberto Nosso Presidente @jairbolsona...,True,"{'tweet_id': 1516929695140429826, 'like_count'...",EvaCapitao2022,"{'user_id': 1433921319792521217, 'name': 'Eva ...",[],False,False,True,"[senadorhumberto, jairbolsonaro]",None,None,senadorhumberto
4,Wed Apr 20 23:59:34 +0000 2022,Senado covarde...seu título de eleitor esse an...,True,"{'tweet_id': 1516929601750151168, 'like_count'...",MarcoocramAlan,"{'user_id': 70003769, 'name': 'Marcoocram', 'd...",[],False,False,False,None,None,None,None


In [29]:
tt_df.to_csv('tweets.csv', index=False)

## Análise de Rede Social

In [30]:
import pandas as pd
import networkx as nx

In [38]:
tt_df = pd.read_csv('tweets.csv')
tt_df.sample(5)

,created_at,tweet_content,truncated_status,tweet_metadata,author,author_metadata,hashtags,is_retweet,is_quoted_rt,is_reply,user_mentions,retweeted_from,quoted_from,in_reply_to
8327,Fri Apr 15 16:44:48 +0000 2022,#MaiorMotociataDoMundo do melhor presidente qu...,False,"{'tweet_id': 1515008249145311238, 'like_count'...",conectabrasiltt,"{'user_id': 1486263155030384641, 'name': '𝗖𝗼𝗻𝗲...",['MaiorMotociataDoMundo'],False,True,False,NaN,NaN,FabioTalhari,NaN
1603,Fri Apr 15 18:42:33 +0000 2022,RT @AdriTSBC: #MaiorMotociataDoMundo com nosso...,False,"{'tweet_id': 1515037880737009672, 'like_count'...",lucianoluck17,"{'user_id': 235287513, 'name': 'Luciano Olivei...",['MaiorMotociataDoMundo'],True,False,False,['AdriTSBC'],AdriTSBC,NaN,NaN
2287,Fri Apr 15 18:30:43 +0000 2022,RT @Brasil8128: O Único que consegue carregar ...,False,"{'tweet_id': 1515034902701305856, 'like_count'...",opinioeslivress,"{'user_id': 1476401863222304773, 'name': 'Opin...",['BolsonaroReeleitoEm2022'],True,False,False,"['Brasil8128', 'LulaOficial']",Brasil8128,NaN,NaN
10689,Fri Apr 15 15:57:22 +0000 2022,RT @MarlyMms: @n_bednarski A esquerdalha mal s...,False,"{'tweet_id': 1514996312776876034, 'like_count'...",Tatiana3170,"{'user_id': 1479478002295660548, 'name': 'Tati...",['MaiorMotociataDoMundo'],True,False,False,"['MarlyMms', 'n_bednarski']",MarlyMms,NaN,NaN
10213,Fri Apr 15 16:13:06 +0000 2022,RT @PileccoMarivane: #MaiorMotociataDoMundo\nÉ...,False,"{'tweet_id': 1515000272610803722, 'like_count'...",jeffermanjj,"{'user_id': 1965439644, 'name': 'Jefferson Seb...",['MaiorMotociataDoMundo'],True,False,False,['PileccoMarivane'],PileccoMarivane,NaN,NaN


In [68]:
# criando as listas dos usuários (nós) e interações entre eles (arestas)
# serão utilizados 4 tipos de interações: (i) retweet nativo; (ii) retweet com quote; (iii) menção; e (iv) reply
users = []
interac = []

for idx, tweet in tt_df.iterrows():
    if tweet['author'] not in users:
        users.append(tweet['author'])
    if not pd.isna(tweet['user_mentions']):
        for user in eval(tweet['user_mentions']):
            interac.append((tweet['author'], user, 'mention'))
            if user not in users:
                users.append(user)
    if tweet['is_retweet']:
        interac.append((tweet['author'], tweet['retweeted_from'], 'retweet'))
        if tweet['retweeted_from'] not in users:
            users.append(tweet['retweeted_from'])
    if tweet['is_quoted_rt']:
        interac.append((tweet['author'], tweet['quoted_from'], 'quoted_rt'))
        if tweet['quoted_from'] not in users:
            users.append(tweet['quoted_from'])
    if tweet['is_reply']:
        interac.append((tweet['author'], tweet['in_reply_to'], 'reply'))
        if tweet['in_reply_to'] not in  users:
            if tweet['in_reply_to'] not in users:
                users.append(tweet['in_reply_to'])

print(f'''
Número de usuários: {len(users)}
Número de interações: {len(interac)}
''')



Número de usuários: 4578
Número de interações: 26624



In [69]:
interac_df = pd.DataFrame(columns=['source', 'target', 'interac_type'], data=interac)
interac_df.head()

,source,target,interac_type
0,FlavioBolsonaro,LulaOficial,mention
1,FlavioBolsonaro,LulaOficial,reply
2,apellegrino,MarceloFreixo,mention
3,EvaCapitao2022,senadorhumberto,mention
4,EvaCapitao2022,jairbolsonaro,mention


In [97]:
interac_weight_df = pd.DataFrame(columns=['source', 'target', 'interac_type'], data=list(interac_df.value_counts().index))
interac_weight_df['weight'] = list(interac_df.value_counts().values)
interac_weight_df = interac_weight_df.query('source != target') # removendo as interações de usuários com eles mesmos
interac_weight_df.head(10)

,source,target,interac_type,weight
0,Chebert11,jairbolsonaro,mention,35
1,CesarMo70820605,jairbolsonaro,mention,19
2,CesarMo70820605,MarchaDaFamili1,retweet,16
3,CesarMo70820605,MarchaDaFamili1,mention,16
4,CelsoRi21369829,jairbolsonaro,mention,14
5,DruzianMonica,MarchaDaFamili1,mention,14
6,lisboa_adriel,MarchaDaFamili1,mention,13
7,Roney35790193,brasililha,retweet,13
8,lisboa_adriel,MarchaDaFamili1,retweet,13
9,Roney35790193,brasililha,mention,13


In [98]:
retweet_df = interac_weight_df.query("interac_type == 'retweet'")
retweet_df.shape

(9036, 4)

In [99]:
quote_df = interac_weight_df.query("interac_type == 'quoted_rt'")
quote_df.shape

(388, 4)

In [100]:
mention_df = interac_weight_df.query("interac_type == 'mention'")
mention_df.shape

(12455, 4)

In [101]:
reply_df = interac_weight_df.query("interac_type == 'reply'")
reply_df.shape

(1000, 4)

In [102]:
retweet_df.to_csv('retweet_nwk.csv', index=False)
quote_df.to_csv('quoted_nwk.csv', index=False)
mention_df.to_csv('mention_ntw.csv', index=False)
reply_df.to_csv('reply_ntw.csv', index=False)

### Selecionando dias

In [140]:
from datetime import datetime

# criando função para converter a data para o formato mm-dd-yyyy (string)
def convert_date(date_str: str):
    _date_str = date_str[4:10]+' 2022'
    _date_str = datetime.strptime(_date_str, '%b %d %Y')
    _date_str = _date_str.strftime('%m-%d-%Y')
    return _date_str

In [141]:
tt_df['converted_date'] = tt_df['created_at'].map(lambda x: convert_date(x))
tt_df['converted_date'] = pd.to_datetime(tt_df['converted_date'])
tt_df.sample(3)

,created_at,tweet_content,truncated_status,tweet_metadata,author,author_metadata,hashtags,is_retweet,is_quoted_rt,is_reply,user_mentions,retweeted_from,quoted_from,in_reply_to,converted_date
4260,Fri Apr 15 17:52:08 +0000 2022,RT @MarizMarcella: Lindo 💚 \n#MaiorMotociataDo...,False,"{'tweet_id': 1515025191440629764, 'like_count'...",bevander_4,"{'user_id': 101320625, 'name': 'BenydeRecifePr...",['MaiorMotociataDoMundo'],True,False,False,['MarizMarcella'],MarizMarcella,NaN,NaN,2022-04-15
4644,Fri Apr 15 17:45:51 +0000 2022,RT @Brasil8128: O Único que consegue carregar ...,False,"{'tweet_id': 1515023612746547205, 'like_count'...",StevensonSoare4,"{'user_id': 1377558167136129025, 'name': 'Stev...",['BolsonaroReeleitoEm2022'],True,False,False,"['Brasil8128', 'LulaOficial']",Brasil8128,NaN,NaN,2022-04-15
11491,Fri Apr 15 15:12:14 +0000 2022,"RT @mechamojunior: O homem tá REELEITO! Deus,o...",False,"{'tweet_id': 1514984954115117062, 'like_count'...",bbmedeiros2,"{'user_id': 1275734646, 'name': 'Bárbara 🇧🇷2️⃣...",['BolsonaroReeleitoEm2022'],True,False,False,"['mechamojunior', 'jairbolsonaro']",mechamojunior,NaN,NaN,2022-04-15


In [158]:
# selecionando dias 14, 15, 16 e 17 de abril

# criando mascara
bool_mask = (tt_df['converted_date']>='2022-04-14') & (tt_df['converted_date'] <= '2022-04-18')
tt_selected_date = tt_df.loc[bool_mask].sort_values(by='converted_date')

In [159]:
tt_selected_date.head()

,created_at,tweet_content,truncated_status,tweet_metadata,author,author_metadata,hashtags,is_retweet,is_quoted_rt,is_reply,user_mentions,retweeted_from,quoted_from,in_reply_to,converted_date
12645,Thu Apr 14 00:14:41 +0000 2022,RT @MARINESARALDI: Em 2018 Eu Defendia Muito O...,False,"{'tweet_id': 1514396690232578055, 'like_count'...",GersonCarioca1,"{'user_id': 1358188575200866305, 'name': 'Gers...",[],True,False,False,['MARINESARALDI'],MARINESARALDI,NaN,NaN,2022-04-14
12404,Thu Apr 14 19:06:06 +0000 2022,RT @AraldiRocha: Nunca Mais Vamos Deixar A Esq...,False,"{'tweet_id': 1514681420794380309, 'like_count'...",printabr,"{'user_id': 17782770, 'name': '22em2022', 'des...",['BolsonaroReeleitoEm2022'],True,False,False,['AraldiRocha'],AraldiRocha,NaN,NaN,2022-04-14
12405,Thu Apr 14 19:03:44 +0000 2022,RT @AraldiRocha: Nunca Mais Vamos Deixar A Esq...,False,"{'tweet_id': 1514680824565841930, 'like_count'...",WilsonmarcosAdv,"{'user_id': 2243221405, 'name': 'Wilson Marcos...",['BolsonaroReeleitoEm2022'],True,False,False,['AraldiRocha'],AraldiRocha,NaN,NaN,2022-04-14
12406,Thu Apr 14 18:43:24 +0000 2022,RT @MARINESARALDI: A Verdadeira Pesquisa Data ...,False,"{'tweet_id': 1514675708030857224, 'like_count'...",carvalholeonic3,"{'user_id': 819702842822946816, 'name': 'Leoni...",['BolsonaroReeleitoEm2022'],True,False,False,['MARINESARALDI'],MARINESARALDI,NaN,NaN,2022-04-14
12407,Thu Apr 14 18:35:08 +0000 2022,RT @AraldiRocha: Nunca Mais Vamos Deixar A Esq...,False,"{'tweet_id': 1514673628578783234, 'like_count'...",GhostWildRecon,"{'user_id': 1512587296821895174, 'name': 'Ghos...",['BolsonaroReeleitoEm2022'],True,False,False,['AraldiRocha'],AraldiRocha,NaN,NaN,2022-04-14


In [160]:
tt_selected_date.tail()

,created_at,tweet_content,truncated_status,tweet_metadata,author,author_metadata,hashtags,is_retweet,is_quoted_rt,is_reply,user_mentions,retweeted_from,quoted_from,in_reply_to,converted_date
4226,Fri Apr 15 17:52:40 +0000 2022,RT @candidobueno: Esse é o cara!!!\n\n#MaiorMo...,False,"{'tweet_id': 1515025326401007616, 'like_count'...",SilvaMarcosrj,"{'user_id': 4019216549, 'name': 'Marcos FlaTT'...",['MaiorMotociataDoMundo'],True,False,False,['candidobueno'],candidobueno,NaN,NaN,2022-04-15
4227,Fri Apr 15 17:52:36 +0000 2022,RT @Marlenepiazza2: #MaiorMotociataDoMundo é o...,False,"{'tweet_id': 1515025310030483465, 'like_count'...",AG21D,"{'user_id': 1142224857364611073, 'name': 'CONS...",['MaiorMotociataDoMundo'],True,False,False,['Marlenepiazza2'],Marlenepiazza2,NaN,NaN,2022-04-15
4217,Fri Apr 15 17:52:45 +0000 2022,RT @RaulitaF: @SF_Moro Bom dia\n#MaiorMotociat...,False,"{'tweet_id': 1515025348467077129, 'like_count'...",bevander_4,"{'user_id': 101320625, 'name': 'BenydeRecifePr...",['MaiorMotociataDoMundo'],True,False,False,"['RaulitaF', 'SF_Moro']",RaulitaF,NaN,NaN,2022-04-15
6328,Fri Apr 15 17:14:13 +0000 2022,RT @AdriTSBC: Gente 😂\nAlgo errado não está ce...,False,"{'tweet_id': 1515015649399029767, 'like_count'...",bbmedeiros2,"{'user_id': 1275734646, 'name': 'Bárbara 🇧🇷2️⃣...",[],True,False,False,['AdriTSBC'],AdriTSBC,NaN,NaN,2022-04-15
2,Sat Apr 16 13:16:16 +0000 2022,O jingle da #ViradaRJ não sai da minha cabeça ...,True,"{'tweet_id': 1515318156977491972, 'like_count'...",apellegrino,"{'user_id': 2689114595, 'name': 'Antonia Pelle...",['ViradaRJ'],False,False,False,['MarceloFreixo'],NaN,NaN,NaN,2022-04-16


In [161]:
tt_selected_date.shape

(12634, 15)

In [162]:
tt_df.shape

(12742, 15)

A grande maioria dos tweets coletados foram dentro das datas selecionadas

In [166]:
G = nx.DiGraph()
G.add_nodes_from(users)

In [214]:
retweet_df.columns

Index(['source', 'target', 'interac_type', 'weight'], dtype='object')

In [170]:
network_list = [retweet_df, quote_df, mention_df, reply_df]

for network in network_list:
    for idx, row in network.iterrows():
        G.add_edge(row['source'], row['target'], weight=row['weight'], interac_type=row['interac_type'])

In [199]:
from random import sample
sample(list(G.edges(data=True)), k=5)

[('CesarMo70820605', 'alos07b3', {'weight': 2, 'interac_type': 'mention'}),
 ('mariahs058', 'jairbolsonaro', {'weight': 1, 'interac_type': 'mention'}),
 ('andre_tamarindo', 'Metropoles', {'weight': 1, 'interac_type': 'quoted_rt'}),
 ('rodrigo_ataides',
  'veramagalhaes',
  {'weight': 1, 'interac_type': 'mention'}),
 ('celmagnani', 'bebadopensador', {'weight': 1, 'interac_type': 'mention'})]

In [205]:
G_retweet = nx.DiGraph(((s, t, d) for s,t,d in G.edges(data=True) if d['interac_type']=='retweet'))
G_quote = nx.DiGraph(((s, t, d) for s,t,d in G.edges(data=True) if d['interac_type']=='quoted_rt'))
G_mention = nx.DiGraph(((s, t, d) for s,t,d in G.edges(data=True) if d['interac_type']=='mention'))
G_reply = nx.DiGraph(((s, t, d) for s,t,d in G.edges(data=True) if d['interac_type']=='reply'))

In [218]:
sorted(G_retweet.in_degree(weight='weight'), key=lambda x: x[1], reverse=True)[:15]

[('MarianaAmary2', 305),
 ('ruivanaro', 19),
 ('O_SPL1NTER', 13),
 ('OdairJosefrSEP', 6),
 ('RtbMoyses', 0),
 ('Kleber01360813', 0),
 ('TeodoraCoimbra', 0),
 ('Lindasod', 0),
 ('CesarMo70820605', 0),
 ('Marlenepiazza2', 0),
 ('Elisiobarreto', 0),
 ('MarchaDaFamili1', 0),
 ('EdsonBertoluci', 0),
 ('silva_debie', 0),
 ('opinioeslivress', 0)]

In [211]:
sorted(G_quote.in_degree, key=lambda x: x[1], reverse=True)[:15]

[('jairbolsonaro', 21),
 ('PATRlOTAS', 11),
 ('AragaoMosart', 10),
 ('MaxGuilhermeOfc', 9),
 ('DEPAULLA', 9),
 ('MarchaDaFamili1', 8),
 ('freu_rodrigues', 8),
 ('Jnfigueiredo39', 8),
 ('LucianoHangBr', 7),
 ('brom_elisa', 7),
 ('yasdositeBr', 7),
 ('MarianaAmary2', 6),
 ('deconauta', 6),
 ('yuriaragao_', 6),
 ('CarlaZambelli38', 5)]

In [219]:
sorted(G_mention.in_degree(weight='weight'), key=lambda x: x[1], reverse=True)[:15]

[('MarchaDaFamili1', 962),
 ('jairbolsonaro', 646),
 ('flferronato', 550),
 ('ViWildhagen', 499),
 ('FlviaLeo16', 464),
 ('MARINESARALDI', 444),
 ('madruga0022', 387),
 ('LulaOficial', 359),
 ('Jouberth19', 354),
 ('deconauta', 344),
 ('kicaKiquinha', 279),
 ('Brasil8128', 278),
 ('AraldiRocha', 200),
 ('Wander_Gomes_64', 183),
 ('relatza', 169)]

In [213]:
sorted(G_reply.in_degree, key=lambda x: x[1], reverse=True)[:15]

[('veramagalhaes', 52),
 ('DeniseCToledo8', 45),
 ('senadorhumberto', 27),
 ('OGloboPolitica', 27),
 ('jairbolsonaro', 26),
 ('MarchaDaFamili1', 25),
 ('LulaOficial', 23),
 ('UOLNoticias', 18),
 ('Jouberth19', 15),
 ('erikakokay', 13),
 ('flferronato', 13),
 ('BlogdoNoblat', 11),
 ('Fiuzafirmao2', 10),
 ('FabioTalhari', 10),
 ('taoquei1', 10)]